$$dX_t = A X_t dt + R_1 dW_t$$
$$dY_t = H X_t dt + R_2 dV_t$$

Model settings: (Conditions that the model needs to satsify)
1. Signal of dimension 'dim', and the observtations are in dimension 'dim_o'
2. $x_0|y_0 \sim N(m_0,C_0)$
3. $R_1, R_2$ is symmetric
4. $R_1$ is commutable with $A$
5. $(A+A')$ is invertible
6. For convenience purpose the target function is $\varphi(x)=x^{1}$, where $x=(x^{1},x^{2},...,x^{dim})$

In this code, I will include the $A,R_1,R_2,H,m_0,C_0$ as parameter as well, so that it's easily scalable wrt any dimension dim.
Note that dim needs to be bigger than 1, as the matrix multiplication sign '@' in python doesn't support '*' in one dimension.

In [4]:
import numpy as np
import matplotlib.pyplot as plt 
import progressbar
from scipy import linalg as la
from scipy.sparse import identity
from scipy.sparse import rand
from scipy.sparse import diags
from scipy.sparse import triu

def gen_model(dim):
    ## dim is dimension value
    dim_o=dim
    A1 = -rand(dim,dim,density=0.75).toarray()/5
    A2 = triu(A1, k=1).toarray()/(10)
    A = diags(np.random.normal(-0.5,0,dim),0).toarray()/50 + A2 - A2.T
    ## we denote R1^{1/2},R2^{1/2} as R1,R2 repectively for convenience
    ## Non-Identity covariance matrix
    R1=(identity(dim).toarray() + np.tri(dim,dim,1) - np.tri(dim,dim,-2))/2
    R2=(identity(dim_o).toarray() + np.tri(dim_o,dim_o,1) - np.tri(dim_o,dim_o,-2))/2

    H=rand(dim_o,dim,density=0.75).toarray()/20
    m0=np.zeros(dim)+6
    C0=identity(dim).toarray()
    ## Collection of input 
    collection_input = [dim,dim_o,A,R1,R2,H,m0,C0]
    return collection_input

def gen_data(T,l,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**l)
    I=identity(dim).toarray()
    tau=2**(-l)
    L=la.expm(A*tau)
    ## We are going to need W to be symmetric! 
    W=(R1@R1)@(la.inv(A+A.T)@(L@(L.T)-I))
    C=tau*H
    V=(R2@R2)*tau

    v=np.zeros((J+1,dim,1))
    z=np.zeros((J+1,dim_o,1))
    v[0]=np.random.multivariate_normal(m0,C0,(1)).T
    z[0]=np.zeros((dim_o,1))


    for j in range(J):
        ## truth
        v[j+1] = L@v[j] + np.random.multivariate_normal(np.zeros(dim),W,(1)).T
        ## observation
        z[j+1] = z[j] + C@v[j+1] + np.random.multivariate_normal(np.zeros(dim_o),V,(1)).T
        
    return([z,v])

def cut(T,lmax,l,v):
    ind = np.arange(T*2**l+1)
    rtau = 2**(lmax-l)
    w = v[ind*rtau]
    return(w)

def EnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**l)
    I=identity(dim).toarray()
    I_o=identity(dim_o).toarray()
    dt=2**(-l)
    
    m=np.zeros((J+1,dim,1))
    c=np.zeros((J+1,dim,dim))
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    for j in range(J):
        dw = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
        dv = np.random.multivariate_normal(np.zeros(dim_o),dt*I_o,N).T
        ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
        ## x-m[j]:dim*N c[j]:dim*dim
        
        step1=(((x-m[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        step3=step2@( (z[j+1]-z[j]) - (H@x*dt + R2@dv) )
        step4=(x-m[j])@step3 /(N-1)
        
        x = x + A@x*dt + R1@dw + step4
        m[j+1] = (np.mean(x, axis=1)).reshape(dim,1)

    return([m,c])

def CEnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**(l-1))
    I=identity(dim).toarray()
    I1=identity(dim_o).toarray()
    dt=2**(-l)
    dt1=2**(-l+1)
    
    m=np.zeros((J*2+1,dim,1))
    m1=np.zeros((J+1,dim,1))
    c=np.zeros((J*2+1,dim,dim))
    c1=np.zeros((J+1,dim,dim))
    z1=cut(T,lmax,l-1,z)
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    x1 = x
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    m1[0]=m[0]
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    c1[0]=c[0]
    
    dw=np.zeros((2,dim,N))
    dv=np.zeros((2,dim_o,N))
    for j in range(J):
        for s in range(2):
            dw[s] = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
            dv[s] = np.random.multivariate_normal(np.zeros(dim_o),dt*I1,N).T
            ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
            ## x-m[j]:dim*N c[j]:dim*dim

            step1=(((x-m[2*j+s]).T)@(H.T))
            step2=step1@(la.inv(R2)@la.inv(R2))
            step3=step2@( (z[2*j+s+1]-z[2*j+s]) - (H@x*dt + R2@dv[s]) )
            step4=(x-m[2*j+s])@step3 /(N-1)

            x = x + A@x*dt + R1@dw[s] + step4
            m[2*j+s+1] = (np.mean(x, axis=1)).reshape(dim,1)
        
        step1=(((x1-m1[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        step3=step2@( (z1[j+1]-z1[j]) - (H@x1*dt1 + R2@(dv[0]+dv[1])) )
        step4=(x1-m1[j])@step3 /(N-1)
        
        x1 = x1 + A@x1*dt1 + R1@(dw[0]+dw[1]) + step4
        m1[j+1] = (np.mean(x1, axis=1)).reshape(dim,1)
        
    return([m,m1])

def DEnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**l)
    I=identity(dim).toarray()
    I_o=identity(dim_o).toarray()
    dt=2**(-l)
    
    m=np.zeros((J+1,dim,1))
    c=np.zeros((J+1,dim,dim))
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    for j in range(J):
        dw = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
        dv = np.random.multivariate_normal(np.zeros(dim_o),dt*I_o,N).T
        ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
        ## x-m[j]:dim*N c[j]:dim*dim
        
        step1=(((x-m[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        # Only the "innovation" term here changes to the "deterministic" version
        step3=step2@( (z[j+1]-z[j]) - (H@(x+m[j])*dt)/2 )
        step4=(x-m[j])@step3 /(N-1)
        
        x = x + A@x*dt + R1@dw + step4
        m[j+1] = (np.mean(x, axis=1)).reshape(dim,1)

    return([m,c])

def DCEnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**(l-1))
    I=identity(dim).toarray()
    I1=identity(dim_o).toarray()
    dt=2**(-l)
    dt1=2**(-l+1)
    
    m=np.zeros((J*2+1,dim,1))
    m1=np.zeros((J+1,dim,1))
    c=np.zeros((J*2+1,dim,dim))
    c1=np.zeros((J+1,dim,dim))
    z1=cut(T,lmax,l-1,z)
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    x1 = x
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    m1[0]=m[0]
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    c1[0]=c[0]
    
    dw=np.zeros((2,dim,N))
    dv=np.zeros((2,dim_o,N))
    for j in range(J):
        for s in range(2):
            dw[s] = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
            dv[s] = np.random.multivariate_normal(np.zeros(dim_o),dt*I1,N).T
            ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
            ## x-m[j]:dim*N c[j]:dim*dim

            step1=(((x-m[2*j+s]).T)@(H.T))
            step2=step1@(la.inv(R2)@la.inv(R2))
            step3=step2@( (z[2*j+s+1]-z[2*j+s]) - (H@(x+m[2*j+s])*dt)/2 )
            step4=(x-m[2*j+s])@step3 /(N-1)

            x = x + A@x*dt + R1@dw[s] + step4
            m[2*j+s+1] = (np.mean(x, axis=1)).reshape(dim,1)
        
        step1=(((x1-m1[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        
        step3=step2@( (z1[j+1]-z1[j]) - (H@(x1+m1[j])*dt1)/2 )
        step4=(x1-m1[j])@step3 /(N-1)
        
        x1 = x1 + A@x1*dt1 + R1@(dw[0]+dw[1]) + step4
        m1[j+1] = (np.mean(x1, axis=1)).reshape(dim,1)
        
    return([m,m1])
  
def coef(x, y): 
    # number of observations/points 
    n = np.size(x) 
  
    # mean of x and y vector 
    m_x, m_y = np.mean(x), np.mean(y) 
  
    # calculating cross-deviation and deviation about x 
    SS_xy = np.sum(y*x) - n*m_y*m_x 
    SS_xx = np.sum(x*x) - n*m_x*m_x 
  
    # calculating regression coefficients 
    b_1 = SS_xy / SS_xx 
    b_0 = m_y - b_1*m_x 
  
    return np.asarray((b_0, b_1)) 

### Paramter Tuning: 
#####  (i) For EnKBF, in order to target a certain MSE value, we need to banlance this two terms above as 
$$
\mathbb{E}[\|\eta_{t}^{N,l}(e)-\eta_{t}(e)\|^2]  = \mathbb{E}[\|\eta_{t}^{N,l}(e) - \eta_{t}^{l}(e)\|^2] + \|\eta_{t}^{l}(e) - \eta_{t}(e)\|^2
$$

From Proposition 2.1 of our paper, we know that EnKBF estimator $\eta_{t}^{N,l}(e)$ satisfies

1. $ \mathbb{E}[\|\eta_{t}^{N,l}(e) - \eta_{t}^{l}(e)\|^2] \approx C_{1}(\frac{1}{N})$

2. $  \|\eta_{t}^{l}(e) - \eta_{t}(e)\| \approx C_{2}(\Delta_{l})$

We simulate to obtain this two constants $C_{1}$, $C_{2}$. Values of $\eta_{t}^{l}(\varphi)$ can be approximated with EnKBF with huge $N$.

##### To target a MSE of $\mathcal{O}(\epsilon^2)$, we choose $N = \epsilon^{-2}C_{1}$, $\Delta_{l} = \frac{\epsilon}{C_{2}}$, the cost required for EnKBF is represented by the number of discretization steps involved
$$
Cost_{enkbf} = N \Delta_{l}^{-1} t = tC_{1}C_{2}\epsilon^{-3} = \mathcal{O}(\epsilon^{-3})
$$

##### (ii) For MLEnKBF,  

Remark: For the models used in the simulation, EnKBF with $l \leq 2$ sometimes output exploding values, for $l >2$ this issue doesn't exist, which suggest that we should consider using for MLEnKBF for $L>3$
$$
\eta_{t}^{ML}(e) = \eta_{t}^{N_{3},3}(e) + \sum_{l=4}^{L} (\eta_{t}^{l}-\eta_{t}^{l-1})^{N_{l}}(e)
$$
to simply avoid getting exploding values from our specific model settings.

The MSE of the MLEnKBF estimator is given by

$$
\mathbb{E}[\|\eta_{t}^{ML}(e)-\eta_{t}(e)\|^2]  = \mathbb{E}[\|\eta_{t}^{ML}(e) - \eta_{t}^{L}(e)\|^2] + \|\eta_{t}^{L}(e) - \eta_{t}(e)\|^2
$$

From Theorem 3.1 of our paper, we know that MLEnKBF estimator satisfies

1. $\mathbb{E}[\|\eta_{t}^{ML}(e) - \eta_{t}^{L}(e)\|^2] \approx \frac{C_{1}}{N_{3}} + C_{3} \sum_{l=4}^{L} \frac{\Delta_{l}}{N_{l}} + C_{4}\sum_{l_{1}=4}^{L}\sum_{l_{2}=4, l_{2} \neq l_{1}}^{L}\frac{\Delta_{l_{1}}}{N_{l_{1}}}\frac{\Delta_{l_{2}}}{N_{l_{2}}} \approx \frac{C_{1}}{N_{3}} + C_{3} \sum_{l=4}^{L} \frac{\Delta_{l}}{N_{l}}$

We remark here that the third term above is in practice much smaller than the first & second term thus can be ignored

In order to tune values of $C_3$, recall that Proposition D.4 gives

2. $\mathbb{E}[\|(\eta_{t}^{l}-\eta_{t}^{l-1})^{N_l}(e) - (\eta_{t}^{l}-\eta_{t}^{l-1})(e)\|^2] \approx C_{3} \frac{\Delta_{l}}{N_{l}}$

We can thus simulate CEnKBF to obtain $C_{3}$ for our specific model, too.

##### To target a MSE of $\mathcal{O}(\epsilon^{2})$ for MLEnKBF , we choose $\Delta_{L} = \frac{\epsilon}{C_{2}}$, $N_{3}=C_{1}\epsilon^{-2}$, $N_{l} = \epsilon^{-2}\Delta_{l}C_{3}(L-3) $ for $l \in \{4,5,...,L\}$, the cost required for MLEnKBF is represented by the number of discretization steps involved
$$
Cost_{mlenkbf} = t\sum_{l=3}^{L}N_{l}\Delta_{l}^{-1} = tC_{1}\Delta_{3}^{-1}\epsilon^{-2} + \sum_{l=4}^{L}C_{3}(L-3)\epsilon^{-2} = \mathcal{O}(\epsilon^{-2}\log(\epsilon)^2)
$$




##### For Determinisitc D-EnKBF/MLD-EnKBF, similar tuning is applied.

In [5]:
### In order to construct (D)MLEnKBF, we will need values of c1,c2,c3 to be fitted out first

In [6]:
def Numcal(C,L):
    Num=np.zeros(L+1)
    epsilon = c2 * 2**(-L)
    Num[3] = int(c1 * epsilon**(-2))
    for li in range(4,L+1):
        Num[li]=int(epsilon**(-2)*c3*(L-3)*2**(-l))
    return(Num*C)

def MLEnKBF(C,T,L,lmax,z,collection_input):
    Num=Numcal(C,L)
    telescopic_summand=np.zeros(L-3+1)
    [m,c]=EnKBF(T,3,lmax,z,int(Num[3]),collection_input)
    telescopic_summand[0]=m[-1,0]
    for l in range(4,L+1):
        [m1,m2]=CEnKBF(T,l,lmax,z,int(Num[l]),collection_input)
        telescopic_summand[l-3]=m1[-1,0]-m2[-1,0]
    est=np.sum(telescopic_summand)
    return(est)

In [7]:
def cost_enkbf(T,L):
    epsilon = c2 * 2**(-L)
    cost=int(T*c1*c2*epsilon**(-3))
    return(cost)

def cost_mlenkbf(C,T,L):
    num=Numcal(C,L)
    kk=0
    for l in range(3,L+1):
        kk=kk+num[l]*(2**(l))
    return(T*kk)

In [8]:
def Numcal_D(C,L):
    Num=np.zeros(L+1)
    epsilon = c2 * 2**(-L)
    Num[3] = int(c1 * epsilon**(-2))
    for li in range(4,L+1):
        Num[li]=int(epsilon**(-2)*c3*(L-3)*2**(-l))
    return(Num*C)

def MLDEnKBF(C,T,L,lmax,z,collection_input):
    Num=Numcal_D(C,L)
    telescopic_summand=np.zeros(L-3+1)
    [m,c]=DEnKBF(T,3,lmax,z,int(Num[3]),collection_input)
    telescopic_summand[0]=m[-1,0]
    for l in range(4,L+1):
        [m1,m2]=DCEnKBF(T,l,lmax,z,int(Num[l]),collection_input)
        telescopic_summand[l-3]=m1[-1,0]-m2[-1,0]
    est=np.sum(telescopic_summand)
    return(est)

In [9]:
def cost_denkbf(T,L):
    epsilon = c2 * 2**(-L)
    cost=int(T*c1*c2*epsilon**(-3))
    return(cost)

def cost_mldenkbf(C,T,L):
    num=Numcal_D(C,L)
    kk=0
    for l in range(3,L+1):
        kk=kk+num[l]*(2**(l))
    return(T*kk)

##### 1. $\mathbb{E}[\|(\eta_{t}^{l,N}(\varphi)-\eta_{t}^{l}(\varphi))\|^2] \approx C_{1}(\frac{1}{N})$
##### 2. $\|\eta_{t}^{l}(e) - \eta_{t}(e)\| \approx C_{2}(\Delta_{l})$
##### 3. $\mathbb{E}[\|(\eta_{t}^{l}-\eta_{t}^{l-1})^{N_l}(e) - (\eta_{t}^{l}-\eta_{t}^{l-1})(e)\|^2] \approx C_{3} \frac{\Delta_{l}}{N_{l}}$
##### 4. Simulation to get MSE of MLEnKBF & EnKBF for $l \in \{4,5,6,7,8,9\}$

In [10]:
def fit_c1(seed_val):
    np.random.seed(seed_val)
    dim = 10
    l=4
    lmax=9
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    num_seq = np.zeros(6)
    for i in range(6):
        num_seq[i] = 10*2**i
        
    Rep=100
    evar=np.zeros(6)
    for numi in range(6):
        est=np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                mean_mat = EnKBF(T,l,lmax,z,int(num_seq[numi]),collection_input)[0]
                est[rep] = np.mean(mean_mat[-1,:])
        evar[numi] = np.var(est)
    
    x = np.log10(num_seq)
    y = np.log10(evar)
    b = coef(x,y)
    return 10**(b[0])

def fit_c2(seed_val):
    np.random.seed(seed_val)
    lmax = 12
    dim = 10
    T=100
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    N=1000
    Rep=100
    tv_rep = np.zeros((Rep,dim))
    for rep in range(Rep):
        tv_rep[rep] = EnKBF(T,lmax,lmax,z,N,collection_input)[0][-1,:][:,0]
    tv_approx = np.mean(tv_rep,axis=0)
    
    bias_level=np.zeros(10)
    delt_level=np.zeros(10)
    for l in range(4,10):
        est=np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                mean_mat = EnKBF(T,l,lmax,z,N,collection_input)[0][-1,:]
                est[rep] = np.sqrt(np.sum((mean_mat - tv_approx)**2))
        bias_level[l] = np.mean(est)
        delt_level[l] = 2**(-l)
    
    x = np.log10(delt_level[4:10])
    y = np.log10(bias_level[4:10])
    b = coef(x,y)
    return 10**(b[0])

def fit_c3(seed_val):
    np.random.seed(seed_val)
    dim = 10
    lmax=9
    N = 50
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    delta_seq = np.zeros(10)
    for l in range(10):
        delta_seq[l] = 2**(-l)
    
    Rep=100
    evar=np.zeros(10)
    for l in range(4,8):
        est = np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                [m,m1]=CEnKBF(T,l,lmax,z,N,collection_input)
                est[rep] = np.mean(m[-1,:]) - np.mean(m1[-1,:])
        evar[l] = np.var(est)
        
    x = np.log10(delta_seq[4:8])
    y = np.log10(evar[4:8])
    b=coef(x,y)
    return N*10**(b[0])

def simulate_mse(seed_val):
    np.random.seed(seed_val)
    lmax = 12
    dim = 10
    T=100
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    N=1000
    
    tv_rep = np.zeros((Rep,dim))
    for rep in range(Rep):
        tv_rep[rep] = EnKBF(T,lmax,lmax,z,N,collection_input)[0][-1,:][:,0]
    tv_approx = np.mean(tv_rep,axis=0)
    mse_level=np.zeros(10)
    delt_level=np.zeros(10)
    C=2.1
    Rep=100
    for l in range(4,10):
        est_ml=np.zeros(Rep)
        est_en=np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                mean_en = EnKBF(T,l,lmax,z,N,collection_input)[0][-1,:]
                mean_ml = MLEnKBF(C,T,L,lmax,z,collection_input)
                est_ml[rep] = np.sum((mean_ml - tv_approx)**2)
                est_en[rep] = np.sum((mean_en - tv_approx)**2)
        mse_ml[l] = np.mean(est_ml)
        mse_en[l] = np.mean(est_en)
    
    return mse_ml[4:10],mse_en[4:10]

##### 1. $\mathbb{E}[\|(\eta_{t}^{l,N}(\varphi)-\eta_{t}^{l}(\varphi))\|^2] \approx C_{1}(\frac{1}{N})$
##### 2. $\|\eta_{t}^{l}(e) - \eta_{t}(e)\| \approx C_{2}(\Delta_{l})$
##### 3. $\mathbb{E}[\|(\eta_{t}^{l}-\eta_{t}^{l-1})^{N_l}(e) - (\eta_{t}^{l}-\eta_{t}^{l-1})(e)\|^2] \approx C_{3} \frac{\Delta_{l}}{N_{l}}$
##### 4. Simulation to get MSE of D-MLEnKBF & D-EnKBF for $l \in \{4,5,6,7,8,9\}$

In [ ]:
def fit_c1(seed_val):
    np.random.seed(seed_val)
    dim = 10
    l=4
    lmax=9
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    num_seq = np.zeros(6)
    for i in range(6):
        num_seq[i] = 10*2**i
        
    Rep=100
    evar=np.zeros(6)
    for numi in range(6):
        est=np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                mean_mat = DEnKBF(T,l,lmax,z,int(num_seq[numi]),collection_input)[0]
                est[rep] = np.mean(mean_mat[-1,:])
        evar[numi] = np.var(est)
    
    x = np.log10(num_seq)
    y = np.log10(evar)
    b = coef(x,y)
    return 10**(b[0])

def fit_c2(seed_val):
    np.random.seed(seed_val)
    lmax = 12
    dim = 10
    T=100
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    N=1000
    Rep=100
    tv_rep = np.zeros((Rep,dim))
    for rep in range(Rep):
        tv_rep[rep] = EnKBF(T,lmax,lmax,z,N,collection_input)[0][-1,:][:,0]
    tv_approx = np.mean(tv_rep,axis=0)
    
    bias_level=np.zeros(10)
    delt_level=np.zeros(10)
    for l in range(4,10):
        est=np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                mean_mat = DEnKBF(T,l,lmax,z,N,collection_input)[0][-1,:]
                est[rep] = np.sqrt(np.sum((mean_mat - tv_approx)**2))
        bias_level[l] = np.mean(est)
        delt_level[l] = 2**(-l)
    
    x = np.log10(delt_level[4:10])
    y = np.log10(bias_level[4:10])
    b = coef(x,y)
    return 10**(b[0])

def fit_c3(seed_val):
    np.random.seed(seed_val)
    dim = 10
    lmax=9
    N = 50
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    delta_seq = np.zeros(10)
    for l in range(10):
        delta_seq[l] = 2**(-l)
    
    Rep=100
    evar=np.zeros(10)
    for l in range(4,8):
        est = np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                [m,m1]=DCEnKBF(T,l,lmax,z,N,collection_input)
                est[rep] = np.mean(m[-1,:]) - np.mean(m1[-1,:])
        evar[l] = np.var(est)
        
    x = np.log10(delta_seq[4:8])
    y = np.log10(evar[4:8])
    b=coef(x,y)
    return N*10**(b[0])

def simulate_mse(seed_val):
    np.random.seed(seed_val)
    lmax = 12
    dim = 10
    T=100
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    N=1000
    
    tv_rep = np.zeros((Rep,dim))
    for rep in range(Rep):
        tv_rep[rep] = EnKBF(T,lmax,lmax,z,N,collection_input)[0][-1,:][:,0]
    tv_approx = np.mean(tv_rep,axis=0)
    mse_level=np.zeros(10)
    delt_level=np.zeros(10)
    C=2.1
    Rep=100
    for l in range(4,10):
        est_ml=np.zeros(Rep)
        est_en=np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                mean_en = DEnKBF(T,l,lmax,z,N,collection_input)[0][-1,:]
                mean_ml = MLDEnKBF(C,T,L,lmax,z,collection_input)
                est_ml[rep] = np.sum((mean_ml - tv_approx)**2)
                est_en[rep] = np.sum((mean_en - tv_approx)**2)
        mse_ml[l] = np.mean(est_ml)
        mse_en[l] = np.mean(est_en)
    
    return mse_ml,mse_en